In [1]:
# テキストコーパスをチャンクに分割
with open('code_gate.py', 'r', encoding='utf-8') as f:
    text = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_text(text)

In [ ]:
print(type(texts))
print("--------")
print(len(texts))
print("--------")
print(texts[0])
print("--------")
print(texts[1])

In [ ]:
# パッセージのベクトル化
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large', model_kwargs={'device': 'cpu'})

In [4]:
from langchain_community.vectorstores import FAISS

# データベースの保存
db = FAISS.from_texts(texts, embeddings)
db.save_local('code.db')

In [4]:
from langchain_community.vectorstores import FAISS

# 保存したデータベースの読み込み
db = FAISS.load_local('code.db',embeddings, allow_dangerous_deserialization=True)

In [ ]:
similarity_sample = db.similarity_search("XOR")
print(len(similarity_sample))
print("--------")
print(type(similarity_sample[0]))
print("--------")
print(similarity_sample[0].page_content)

In [6]:
# 検索器の構築
retriever = db.as_retriever()   # 検索文書数 4（デフォルト）

In [ ]:
# モデルの準備
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

#QwenのCoderモデルを使用
model_id = "Qwen/Qwen2.5-Coder-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, legacy=False, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    ).eval()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.01,
    repetition_penalty=2.0,
    )

In [8]:
# プロンプトの準備

template = """
ユーザー:以下のテキストを参照して、それに続く質問に答えてください。

{context}

{question}

システム:"""

from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
    )

In [9]:
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline

qa = RetrievalQA.from_chain_type(
    llm=HuggingFacePipeline(pipeline=pipe),
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
    )

In [ ]:
# 実行例

q = "どんなプログラミング言語で書かれていますか？"
ans = qa.invoke(q)
#print(ans['result'])
import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(q)
print(ans0)

print("--------------------------")

q = "定義されている関数を全てリストアップして下さい"
ans = qa.invoke(q)
#print(ans['result'])
import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(q)
print(ans0)

print("--------------------------")

q = "AND関数は何回呼ばれますか？"
ans = qa.invoke(q)
#print(ans['result'])
import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(q)
print(ans0)

print("--------------------------")

q = "このプログラムがやっていることは何ですか？"
ans = qa.invoke(q)
#print(ans['result'])
import re
pattern = re.compile(r'システム:(.*)',re.DOTALL)
match = pattern.search(ans['result'])
ans0 = match.group(1)
print(q)
print(ans0)
